In [177]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
from scipy import stats
import openpyxl

In [165]:
# Create the dataset
df = pd.read_csv('datasets/brazilian-fuel-dataset.tsv', sep='\t')

In [166]:
# Delete columns COEF DE VARIAÇÃO DISTRIBUIÇÃO and DESVIO PADRÃO DISTRIBUIÇÃO and COEF DE VARIAÇÃO REVENDA and DESVIO PADRÃO REVENDA
df = df.drop(columns=['COEF DE VARIAÇÃO DISTRIBUIÇÃO', 'DESVIO PADRÃO DISTRIBUIÇÃO', 'COEF DE VARIAÇÃO REVENDA', 'DESVIO PADRÃO REVENDA'])

In [167]:
# Trocar formato da data de yyyy/mm/dd para dd/mm/yyyy
df['DATA INICIAL'] = pd.to_datetime(df['DATA INICIAL'])
df['DATA FINAL'] = pd.to_datetime(df['DATA FINAL'])

df['DATA INICIAL'] = df['DATA INICIAL'].dt.strftime('%d/%m/%Y')
df['DATA FINAL'] = df['DATA FINAL'].dt.strftime('%d/%m/%Y')


In [168]:
# Excluir registros que possuem somente "-" como valor
df.replace('-', pd.NA, inplace=True)
df.dropna(inplace=True)

In [169]:
# Transformar as colunas de valores em Float
cols_to_float = ['PREÇO MÉDIO REVENDA', 'PREÇO MÍNIMO REVENDA', 'PREÇO MÁXIMO REVENDA',
                 'MARGEM MÉDIA REVENDA', 'PREÇO MÉDIO DISTRIBUIÇÃO',
                 'PREÇO MÍNIMO DISTRIBUIÇÃO', 'PREÇO MÁXIMO DISTRIBUIÇÃO']
df[cols_to_float] = df[cols_to_float].replace(',', '.', regex=True).astype(float)

In [170]:
# Renomear colunas
df.rename(columns={'DATA INICIAL': 'data_inicial', 'DATA FINAL': 'data_final', 'REGIÃO': 'regiao',
                   'ESTADO': 'estado', 'PRODUTO': 'produto', 'NÚMERO DE POSTOS PESQUISADOS': 'num_postos',
                   'UNIDADE DE MEDIDA': 'unidade', 'PREÇO MÉDIO REVENDA': 'preco_medio_revenda',
                   'PREÇO MÍNIMO REVENDA': 'preco_min_revenda', 'PREÇO MÁXIMO REVENDA': 'preco_max_revenda',
                   'MARGEM MÉDIA REVENDA': 'margem_media_revenda', 'PREÇO MÉDIO DISTRIBUIÇÃO': 'preco_medio_distribuicao',
                   'PREÇO MÍNIMO DISTRIBUIÇÃO': 'preco_min_distribuicao', 'PREÇO MÁXIMO DISTRIBUIÇÃO': 'preco_max_distribuicao'}, inplace=True)

In [171]:
# Alterar os datatypes
df = df.astype({
    'data_inicial': 'object',
    'data_final': 'object',
    'regiao': 'object',
    'estado': 'object',
    'produto': 'object',
    'num_postos': 'int64',
    'unidade': 'object',
    'preco_medio_revenda': 'float64',
    'preco_min_revenda': 'float64',
    'preco_max_revenda': 'float64',
    'margem_media_revenda': 'float64',
    'preco_medio_distribuicao': 'float64',
    'preco_min_distribuicao': 'float64',
    'preco_max_distribuicao': 'float64'
})

In [182]:
# Excluir linhas onde o preco_medio_revenda e margem_media_revenda são -99999.00
df = df[df['preco_medio_revenda'] != -99999.0]
df = df[df['margem_media_revenda'] != -99999.0]


In [181]:
# Exportar o dataset em um arquivo Excel
df.to_excel("./outputs/br_fuel_excel.xlsx", index=False)
